In [1]:
import pandas as pd
import io
import re
from collections import Counter
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()


Saving labeled_text.csv to labeled_text.csv


In [4]:
dataset = pd.read_csv(io.BytesIO(uploaded['labeled_text.csv']))
dataset = dataset[['text','categories']]


In [6]:
dataset = pd.read_csv('labeled_text.csv')
dataset = dataset[['text','categories']]
dataset.head(10)

,text,categories
0,As the deadly Cororavirus spreads across China...,Negative
1,The masks filter out 95 percent of particles a...,Neutral
2,So here’s some moderately good news in the era...,Positive
3,We should also worry about state polls with ve...,Negative
4,“It’s important to emphasise that these are no...,Positive
5,First coronavirus patient discharged from Toro...,Positive
6,"The industry outlook is still profitable, alth...",Positive
7,Here are our most recent articles that mention...,Neutral
8,"“When you order takeaway food, our apartment c...",Negative
9,By Barani Krishnan\nInvesting.com - The corona...,Positive


In [8]:
labels = {"Negative":0, "Neutral":1, "Positive":1}
y = np.array([labels[sentiment] for sentiment in list(dataset['categories'])])
y[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 1])

In [9]:
import re, nltk, torch, nltk
import numpy as np
from numpy import asarray, zeros
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim import models

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
from gensim.models import Word2Vec

In [11]:
def clean_str(string):
    """
    String clean up reg exes.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    
    # return lowercase version of entire sentence
    # and remove all leading spaces
    return string.strip().lower()

In [12]:
# Apply the clean_str function to our sentences
cleaned_sentences = []
uncleaned_sentences = []
for sentence in dataset["text"]:
    cleaned_sentences.append(clean_str(sentence.strip()))
    uncleaned_sentences.append(sentence.strip())

In [13]:
cleaned_sentences_tokenized = []
for i in cleaned_sentences:
  text = nltk.word_tokenize(i)
  cleaned_sentences_tokenized.append(text)


In [16]:
max_features = 2000 # As used in the next line, this only keeps the 2000 most frequent words
max_message = 35 # A relatively short message

tokenizer = Tokenizer(num_words = max_features, ) #https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
tokenizer.fit_on_texts(cleaned_sentences)
X = tokenizer.texts_to_sequences(cleaned_sentences)

# It is important to measure the longest message before setting this
X = pad_sequences(X, padding = 'post' ,maxlen=max_message)
Y = pd.get_dummies(dataset['categories']).values

vocab_size = X.max() + 1 # We need to have one extra slot for the OOV/UNK token
vocab_size # Show the vocab size: Should match max_features from above

2000

In [17]:
Y.shape

(18670, 3)

In [18]:
w2v_model = Word2Vec(cleaned_sentences_tokenized, size=100, min_count=1, sg = 0)

In [19]:
words = list(w2v_model.wv.vocab)
print('Vocabulary size: %d' % len(words))
print(words)

Vocabulary size: 28867
['as', 'the', 'deadly', 'cororavirus', 'spreads', 'across', 'china', 'and', 'other', 'parts', 'of', 'world', ',', 'south', 'africans', 'have', 'grown', 'increasingly', 'worried', 'wondering', 'if', 'they', 'are', 'at', 'risk', 'locally', 'how', 'to', 'keep', 'themselves', 'safe', 'from', 'possible', 'infection', 'masks', 'filter', 'out', '95', 'percent', 'particles', 'small', '0', '3', 'microns', 'air', 'so', 'here', 's', 'some', 'moderately', 'good', 'news', 'in', 'era', 'climate', 'change', 'we', 'should', 'also', 'worry', 'about', 'state', 'polls', 'with', 'very', 'sample', 'sizes', 'big', 'gaps', 'coverage', 'it', 'important', 'emphasise', 'that', 'these', 'not', 'gotcha', 'stress', 'tests', 'mr', 'carney', 'said', 'first', 'coronavirus', 'patient', 'discharged', 'toronto', 'hospital', 'release', 'states', 'use', 'cameras', 'will', 'be', 'for', 'monitoring', 'incident', 'management', 'calgary', 'traffic', 'centre', '\\', '(', 'tmc', ')', 'help', 'roadways', '

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

In [21]:

embedding_width = 100
embedding_matrix = np.zeros((vocab_size,embedding_width))
oov_errors = 0

for i in range(1, (tokenizer.num_words-1)):

  word = tokenizer.index_word[i] # This is the string value we want to use for the lookup

  # Here we need to catch KeyErrors so that we can proceed with the loop
  # OOV words are simply left with weights of 0
  try:
    embedding_matrix[i] = w2v_model.wv[word]
    #print(embedding_matrix.shape)
  except KeyError:
    oov_errors += 1


print("Out of vocab errors: ", oov_errors)

Out of vocab errors:  2


In [59]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.48213279, -0.6951434 ,  0.96565628, ..., -2.58429599,
        -0.31728223,  1.82262945],
       [-1.87665403, -0.85249472, -1.94737184, ..., -0.42567995,
        -1.02315366,  0.40367499],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.54960454, -2.53038001, -1.4552325 , ...,  0.13096412,
         1.81923735,  0.95836747],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
from tensorflow import keras # As usual, we will use keras as a front end to TF

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=200, shuffle=False)
print(X.shape)
print(y.shape)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(18670, 35)
(18670,)
(14936, 35)
(3734, 35)
(14936,)
(3734,)


In [24]:
lstm_out=64
model = keras.Sequential()
model.add(Embedding(vocab_size, embedding_width, input_length =X.shape[1], weights = [ embedding_matrix] , trainable = True))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
#model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 35, 100)           200000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 301,250
Trainable params: 301,250
Non-trainable params: 0
__________________________________________________

In [25]:
batch_size = 128
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, y_test))

Epoch 1/5
117/117 [==============================] - 23s 155ms/step - loss: 0.6416 - accuracy: 0.6393 - val_loss: 0.6170 - val_accuracy: 0.6564
Epoch 2/5
117/117 [==============================] - 14s 121ms/step - loss: 0.6098 - accuracy: 0.6633 - val_loss: 0.5794 - val_accuracy: 0.6904
Epoch 3/5
117/117 [==============================] - 14s 121ms/step - loss: 0.5557 - accuracy: 0.7157 - val_loss: 0.5106 - val_accuracy: 0.7571
Epoch 4/5
117/117 [==============================] - 14s 120ms/step - loss: 0.4797 - accuracy: 0.7730 - val_loss: 0.4750 - val_accuracy: 0.7860
Epoch 5/5
117/117 [==============================] - 14s 123ms/step - loss: 0.4194 - accuracy: 0.8169 - val_loss: 0.4673 - val_accuracy: 0.7962


In [26]:
model = keras.Sequential()
model.add(Embedding(vocab_size, embedding_width, input_length =X.shape[1], weights = [ embedding_matrix] , trainable = True))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
#model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()
batch_size = 128
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 35, 100)           200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 301,250
Trainable params: 301,250
Non-trainable params: 0
________________________________________________

In [27]:
!wget https://ist664yash.s3.amazonaws.com/project+data/review.csv
a = pd.read_csv('review.csv')

--2022-05-09 00:54:22--  https://ist664yash.s3.amazonaws.com/project+data/review.csv
Resolving ist664yash.s3.amazonaws.com (ist664yash.s3.amazonaws.com)... 52.216.187.83
Connecting to ist664yash.s3.amazonaws.com (ist664yash.s3.amazonaws.com)|52.216.187.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 670320703 (639M) [text/csv]
Saving to: ‘review.csv’

review.csv          100%[===================>] 639.27M  89.5MB/s    in 7.2s    

2022-05-09 00:54:30 (88.4 MB/s) - ‘review.csv’ saved [670320703/670320703]



In [28]:
b = a['text'][0]
text =[]
b = a[['text']]

In [29]:
test_sentences = []
for sentence in b['text']:
  test_sentences.append(sentence)

In [30]:
test_list = test_sentences
X_t = tokenizer.texts_to_sequences(test_list)
text_list = pad_sequences(X_t, padding = 'post' ,maxlen=max_message)

In [41]:
t = model.predict(text_list)
t0 = []
t1 = []
sentiment = []

for i in t:
  if i[0] > i[1]:
    t0.append(1)
    t1.append(0)
    sentiment.append('Negative')

  if i[1] > i[0]:
    t0.append(0)
    t1.append(1)
    sentiment.append('Positive')
    

In [40]:
t[0]

array([0.16698661, 0.83301336], dtype=float32)

In [37]:
sentiment = []
for i in range(0,len(t)):
  if t0[i] == 1:
    sentiment.append('Negative')
  if t1[1] == 1:
    sentiment.append('Positive')

  


1128531

In [42]:
data_to_csv = {'text':test_sentences, 'sentiment':sentiment}
dataframe = pd.DataFrame(data_to_csv) 
dataframe.to_csv('final_output.csv')